In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import math
import scipy.stats as stats
from scipy.stats import skew, norm
from sklearn.preprocessing import LabelEncoder,StandardScaler


from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

In [14]:
data_w = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [15]:
target = data_w['SalePrice']
test_id = test['Id']
test = test.drop(['Id'],axis = 1)
data_w2 = data_w.drop(['SalePrice'], axis = 1)

In [16]:
train_test = pd.concat([data_w2,test], axis=0, sort=False)
train_test.reset_index(inplace=True)

In [17]:
train_test

,index,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0,1.0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,1,2.0,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,2,3.0,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,3,4.0,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,4,5.0,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,1454,NaN,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2915,1455,NaN,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2916,1456,NaN,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
2917,1457,NaN,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5

In [18]:
train_test['MSSubClass'] = train_test['MSSubClass'].apply(str)
train_test['YrSold'] = train_test['YrSold'].apply(str)
train_test['MoSold'] = train_test['MoSold'].apply(str)

In [19]:
train_test['Functional'] = train_test['Functional'].fillna('Typ')
train_test['Electrical'] = train_test['Electrical'].fillna("SBrkr")
train_test['KitchenQual'] = train_test['KitchenQual'].fillna("TA")
train_test['Exterior1st'] = train_test['Exterior1st'].fillna(train_test['Exterior1st'].mode()[0])
train_test['Exterior2nd'] = train_test['Exterior2nd'].fillna(train_test['Exterior2nd'].mode()[0])
train_test['SaleType'] = train_test['SaleType'].fillna(train_test['SaleType'].mode()[0])
train_test["PoolQC"] = train_test["PoolQC"].fillna("None")
train_test["Alley"] = train_test["Alley"].fillna("None")
train_test['FireplaceQu'] = train_test['FireplaceQu'].fillna("None")
train_test['Fence'] = train_test['Fence'].fillna("None")
train_test['MiscFeature'] = train_test['MiscFeature'].fillna("None")

for col in ('GarageArea', 'GarageCars'):
    train_test[col] = train_test[col].fillna(0)
        
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_test[col] = train_test[col].fillna('None')
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    train_test[col] = train_test[col].fillna('None')

In [20]:
train_test.head()

,index,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0,1.0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal
1,1,2.0,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal
2,2,3.0,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal
3,3,4.0,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml
4,4,5.0,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal


In [21]:
useless = ['GarageYrBlt','YearRemodAdd'] 
train_test = train_test.drop(useless, axis = 1)

In [22]:
def impute_knn(df):
    ttn = train_test.select_dtypes(include=[np.number])
    ttc = train_test.select_dtypes(exclude=[np.number])

    cols_nan = ttn.columns[ttn.isna().any()].tolist()    
    cols_no_nan = ttn.columns.difference(cols_nan).values

    for col in cols_nan:
        imp_test = ttn[ttn[col].isna()]
        imp_train = ttn.dropna()        
        model = KNeighborsRegressor(n_neighbors=5)  
        knr = model.fit(imp_train[cols_no_nan], imp_train[col])
        ttn.loc[ttn[col].isna(), col] = knr.predict(imp_test[cols_no_nan])
    
    return pd.concat([ttn,ttc],axis=1)

In [23]:
train_test = impute_knn(train_test)

In [24]:
objects = []
for i in train_test.columns:
    if train_test[i].dtype == object:
        objects.append(i)
train_test.update(train_test[objects].fillna('None'))

In [25]:
train_test["SqFtPerRoom"] = train_test["GrLivArea"] / (train_test["TotRmsAbvGrd"] +
                                                       train_test["FullBath"] +
                                                       train_test["HalfBath"] +
                                                       train_test["KitchenAbvGr"])

train_test['Total_Home_Quality'] = train_test['OverallQual'] + train_test['OverallCond']

train_test['Total_Bathrooms'] = (train_test['FullBath'] + (0.5 * train_test['HalfBath']) +
                               train_test['BsmtFullBath'] + (0.5 * train_test['BsmtHalfBath']))

train_test["HighQualSF"] = train_test["1stFlrSF"] + train_test["2ndFlrSF"]

In [26]:
train_test['MSSubClass'] = train_test['MSSubClass'].apply(str)
train_test['YrSold'] = train_test['YrSold'].(str)
train_test['MoSold'] = train_test['MoSold'].apply(str)

In [33]:
train_test_dummy.head()

,index,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SqFtPerRoom,Total_Home_Quality,Total_Bathrooms,HighQualSF,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_None,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,Utilities_None,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Function

In [27]:
train_test_dummy = pd.get_dummies(train_test)

In [28]:
numeric_features = train_test_dummy.dtypes[train_test_dummy.dtypes != object].index
skewed_features = train_test_dummy[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_features[skewed_features > 0.5]
skew_index = high_skew.index

In [29]:
for i in skew_index:
    train_test_dummy[i] = np.log1p(train_test_dummy[i])

In [30]:
train = train_test_dummy[0:1460]
test = train_test_dummy[1460:2919]
test['Id'] = test_id

In [32]:
train.shape

(1460, 338)

In [26]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [27]:
def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, train, target_log, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [28]:
target_log = np.log1p(target)

In [29]:
target_log

0       12.247699
1       12.109016
2       12.317171
3       11.849405
4       12.429220
          ...    
1455    12.072547
1456    12.254868
1457    12.493133
1458    11.864469
1459    11.901590
Name: SalePrice, Length: 1460, dtype: float64

In [30]:
X_train,X_val,y_train,y_val = train_test_split(train,target_log,test_size = 0.1,random_state=42)

In [75]:
cat = CatBoostRegressor()
cat_model = cat.fit(X_train,y_train,
                     eval_set = (X_val,y_val),
                     plot=True,
                     verbose = 0)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [78]:
cat_pred = cat_model.predict(X_val)
cat_score = rmse(y_val, cat_pred)
cat_score

0.11051329532673733

In [79]:
grid = {'iterations': [1000,6000],
        'learning_rate': [0.05, 0.005, 0.0005],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 9]}

final_model = CatBoostRegressor()
randomized_search_result = final_model.randomized_search(grid,
                                                   X = X_train,
                                                   y= y_train,
                                                   verbose = False,
                                                   plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 12.0307556	test: 12.0060979	best: 12.0060979 (0)	total: 1.32ms	remaining: 1.32s
1:	learn: 12.0247924	test: 12.0001207	best: 12.0001207 (1)	total: 2.76ms	remaining: 1.38s
2:	learn: 12.0188270	test: 11.9941486	best: 11.9941486 (2)	total: 6.7ms	remaining: 2.23s
3:	learn: 12.0128677	test: 11.9881922	best: 11.9881922 (3)	total: 8.33ms	remaining: 2.07s
4:	learn: 12.0068736	test: 11.9821976	best: 11.9821976 (4)	total: 9.02ms	remaining: 1.79s
5:	learn: 12.0009174	test: 11.9762405	best: 11.9762405 (5)	total: 10.3ms	remaining: 1.71s
6:	learn: 11.9949516	test: 11.9702698	best: 11.9702698 (6)	total: 11.6ms	remaining: 1.65s
7:	learn: 11.9890064	test: 11.9643460	best: 11.9643460 (7)	total: 13.4ms	remaining: 1.67s
8:	learn: 11.9830683	test: 11.9584113	best: 11.9584113 (8)	total: 15.2ms	remaining: 1.67s
9:	learn: 11.9771190	test: 11.9524481	best: 11.9524481 (9)	total: 16.7ms	remaining: 1.65s
10:	learn: 11.9711948	test: 11.9465255	best: 11.9465255 (10)	total: 18.2ms	remaining: 1.64s
11:	learn

206:	learn: 10.8621835	test: 10.8390799	best: 10.8390799 (206)	total: 195ms	remaining: 747ms
207:	learn: 10.8568029	test: 10.8336994	best: 10.8336994 (207)	total: 196ms	remaining: 746ms
208:	learn: 10.8514325	test: 10.8283137	best: 10.8283137 (208)	total: 197ms	remaining: 745ms
209:	learn: 10.8460685	test: 10.8229426	best: 10.8229426 (209)	total: 198ms	remaining: 746ms
210:	learn: 10.8407158	test: 10.8175885	best: 10.8175885 (210)	total: 200ms	remaining: 747ms
211:	learn: 10.8353216	test: 10.8121947	best: 10.8121947 (211)	total: 201ms	remaining: 746ms
212:	learn: 10.8299478	test: 10.8068325	best: 10.8068325 (212)	total: 202ms	remaining: 745ms
213:	learn: 10.8245918	test: 10.8015216	best: 10.8015216 (213)	total: 203ms	remaining: 747ms
214:	learn: 10.8192222	test: 10.7961597	best: 10.7961597 (214)	total: 204ms	remaining: 746ms
215:	learn: 10.8138558	test: 10.7908064	best: 10.7908064 (215)	total: 205ms	remaining: 746ms
216:	learn: 10.8085324	test: 10.7854908	best: 10.7854908 (216)	total: 

408:	learn: 9.8270333	test: 9.8056139	best: 9.8056139 (408)	total: 389ms	remaining: 562ms
409:	learn: 9.8221576	test: 9.8007278	best: 9.8007278 (409)	total: 390ms	remaining: 561ms
410:	learn: 9.8173009	test: 9.7958612	best: 9.7958612 (410)	total: 392ms	remaining: 561ms
411:	learn: 9.8124303	test: 9.7910045	best: 9.7910045 (411)	total: 392ms	remaining: 560ms
412:	learn: 9.8075765	test: 9.7861532	best: 9.7861532 (412)	total: 393ms	remaining: 559ms
413:	learn: 9.8027265	test: 9.7813133	best: 9.7813133 (413)	total: 394ms	remaining: 558ms
414:	learn: 9.7978900	test: 9.7764809	best: 9.7764809 (414)	total: 395ms	remaining: 557ms
415:	learn: 9.7930394	test: 9.7716346	best: 9.7716346 (415)	total: 396ms	remaining: 556ms
416:	learn: 9.7881912	test: 9.7667989	best: 9.7667989 (416)	total: 397ms	remaining: 555ms
417:	learn: 9.7833395	test: 9.7619394	best: 9.7619394 (417)	total: 398ms	remaining: 554ms
418:	learn: 9.7784639	test: 9.7570680	best: 9.7570680 (418)	total: 399ms	remaining: 553ms
419:	learn

625:	learn: 8.8246199	test: 8.8048034	best: 8.8048034 (625)	total: 583ms	remaining: 348ms
626:	learn: 8.8202386	test: 8.8004177	best: 8.8004177 (626)	total: 584ms	remaining: 347ms
627:	learn: 8.8158572	test: 8.7960384	best: 8.7960384 (627)	total: 585ms	remaining: 346ms
628:	learn: 8.8114888	test: 8.7916632	best: 8.7916632 (628)	total: 586ms	remaining: 345ms
629:	learn: 8.8071182	test: 8.7872938	best: 8.7872938 (629)	total: 587ms	remaining: 345ms
630:	learn: 8.8027309	test: 8.7829060	best: 8.7829060 (630)	total: 588ms	remaining: 344ms
631:	learn: 8.7983651	test: 8.7785860	best: 8.7785860 (631)	total: 589ms	remaining: 343ms
632:	learn: 8.7939952	test: 8.7742347	best: 8.7742347 (632)	total: 590ms	remaining: 342ms
633:	learn: 8.7896318	test: 8.7698757	best: 8.7698757 (633)	total: 591ms	remaining: 341ms
634:	learn: 8.7852749	test: 8.7655203	best: 8.7655203 (634)	total: 592ms	remaining: 340ms
635:	learn: 8.7809077	test: 8.7611481	best: 8.7611481 (635)	total: 593ms	remaining: 340ms
636:	learn

...

4037:	learn: 0.0708141	test: 0.1372781	best: 0.1372781 (4037)
4038:	learn: 0.0708067	test: 0.1372748	best: 0.1372748 (4038)
4039:	learn: 0.0707949	test: 0.1372730	best: 0.1372730 (4039)
4040:	learn: 0.0707832	test: 0.1372731	best: 0.1372730 (4039)
4041:	learn: 0.0707775	test: 0.1372699	best: 0.1372699 (4041)
4042:	learn: 0.0707659	test: 0.1372687	best: 0.1372687 (4042)
4043:	learn: 0.0707572	test: 0.1372677	best: 0.1372677 (4043)
4044:	learn: 0.0707507	test: 0.1372652	best: 0.1372652 (4044)
4045:	learn: 0.0707410	test: 0.1372613	best: 0.1372613 (4045)
4046:	learn: 0.0707331	test: 0.1372562	best: 0.1372562 (4046)
4047:	learn: 0.0707222	test: 0.1372564	best: 0.1372562 (4046)
4048:	learn: 0.0707128	test: 0.1372511	best: 0.1372511 (4048)
4049:	learn: 0.0707065	test: 0.1372486	best: 0.1372486 (4049)
4050:	learn: 0.0706957	test: 0.1372493	best: 0.1372486 (4049)
4051:	learn: 0.0706845	test: 0.1372491	best: 0.1372486 (4049)
4052:	learn: 0.0706784	test: 0.1372480	best: 0.1372480 (4052)
4053:	le

4448:	learn: 0.0669137	test: 0.1369655	best: 0.1369655 (4448)
4449:	learn: 0.0669047	test: 0.1369603	best: 0.1369603 (4449)
4450:	learn: 0.0668964	test: 0.1369570	best: 0.1369570 (4450)
4451:	learn: 0.0668889	test: 0.1369551	best: 0.1369551 (4451)
4452:	learn: 0.0668805	test: 0.1369570	best: 0.1369551 (4451)
4453:	learn: 0.0668719	test: 0.1369560	best: 0.1369551 (4451)
4454:	learn: 0.0668625	test: 0.1369551	best: 0.1369551 (4454)
4455:	learn: 0.0668508	test: 0.1369563	best: 0.1369551 (4454)
4456:	learn: 0.0668411	test: 0.1369513	best: 0.1369513 (4456)
4457:	learn: 0.0668356	test: 0.1369514	best: 0.1369513 (4456)
4458:	learn: 0.0668279	test: 0.1369508	best: 0.1369508 (4458)
4459:	learn: 0.0668115	test: 0.1369532	best: 0.1369508 (4458)
4460:	learn: 0.0668045	test: 0.1369504	best: 0.1369504 (4460)
4461:	learn: 0.0667931	test: 0.1369521	best: 0.1369504 (4460)
4462:	learn: 0.0667837	test: 0.1369508	best: 0.1369504 (4460)
4463:	learn: 0.0667737	test: 0.1369533	best: 0.1369504 (4460)
4464:	le

5055:	learn: 0.0617517	test: 0.1367243	best: 0.1367243 (5055)
5056:	learn: 0.0617427	test: 0.1367199	best: 0.1367199 (5056)
5057:	learn: 0.0617343	test: 0.1367230	best: 0.1367199 (5056)
5058:	learn: 0.0617267	test: 0.1367266	best: 0.1367199 (5056)
5059:	learn: 0.0617212	test: 0.1367244	best: 0.1367199 (5056)
5060:	learn: 0.0617116	test: 0.1367202	best: 0.1367199 (5056)
5061:	learn: 0.0617004	test: 0.1367190	best: 0.1367190 (5061)
5062:	learn: 0.0616926	test: 0.1367209	best: 0.1367190 (5061)
5063:	learn: 0.0616845	test: 0.1367217	best: 0.1367190 (5061)
5064:	learn: 0.0616780	test: 0.1367210	best: 0.1367190 (5061)
5065:	learn: 0.0616707	test: 0.1367228	best: 0.1367190 (5061)
5066:	learn: 0.0616630	test: 0.1367221	best: 0.1367190 (5061)
5067:	learn: 0.0616559	test: 0.1367200	best: 0.1367190 (5061)
5068:	learn: 0.0616470	test: 0.1367203	best: 0.1367190 (5061)
5069:	learn: 0.0616392	test: 0.1367200	best: 0.1367190 (5061)
5070:	learn: 0.0616333	test: 0.1367177	best: 0.1367177 (5070)
5071:	le

5573:	learn: 0.0578268	test: 0.1365265	best: 0.1365265 (5573)
5574:	learn: 0.0578175	test: 0.1365266	best: 0.1365265 (5573)
5575:	learn: 0.0578107	test: 0.1365255	best: 0.1365255 (5575)
5576:	learn: 0.0578027	test: 0.1365224	best: 0.1365224 (5576)
5577:	learn: 0.0577961	test: 0.1365230	best: 0.1365224 (5576)
5578:	learn: 0.0577866	test: 0.1365189	best: 0.1365189 (5578)
5579:	learn: 0.0577800	test: 0.1365161	best: 0.1365161 (5579)
5580:	learn: 0.0577722	test: 0.1365152	best: 0.1365152 (5580)
5581:	learn: 0.0577677	test: 0.1365136	best: 0.1365136 (5581)
5582:	learn: 0.0577590	test: 0.1365141	best: 0.1365136 (5581)
5583:	learn: 0.0577527	test: 0.1365115	best: 0.1365115 (5583)
5584:	learn: 0.0577449	test: 0.1365117	best: 0.1365115 (5583)
5585:	learn: 0.0577363	test: 0.1365132	best: 0.1365115 (5583)
5586:	learn: 0.0577284	test: 0.1365116	best: 0.1365115 (5583)
5587:	learn: 0.0577217	test: 0.1365102	best: 0.1365102 (5587)
5588:	learn: 0.0577135	test: 0.1365097	best: 0.1365097 (5588)
5589:	le

5809:	learn: 0.0561234	test: 0.1364985	best: 0.1364898 (5779)
5810:	learn: 0.0561152	test: 0.1364981	best: 0.1364898 (5779)
5811:	learn: 0.0561078	test: 0.1364996	best: 0.1364898 (5779)
5812:	learn: 0.0561024	test: 0.1364973	best: 0.1364898 (5779)
5813:	learn: 0.0560961	test: 0.1364983	best: 0.1364898 (5779)
5814:	learn: 0.0560885	test: 0.1364989	best: 0.1364898 (5779)
5815:	learn: 0.0560814	test: 0.1364998	best: 0.1364898 (5779)
5816:	learn: 0.0560735	test: 0.1364985	best: 0.1364898 (5779)
5817:	learn: 0.0560667	test: 0.1364978	best: 0.1364898 (5779)
5818:	learn: 0.0560596	test: 0.1364999	best: 0.1364898 (5779)
5819:	learn: 0.0560500	test: 0.1365003	best: 0.1364898 (5779)
5820:	learn: 0.0560435	test: 0.1364975	best: 0.1364898 (5779)
5821:	learn: 0.0560372	test: 0.1365006	best: 0.1364898 (5779)
5822:	learn: 0.0560296	test: 0.1365010	best: 0.1364898 (5779)
5823:	learn: 0.0560238	test: 0.1364988	best: 0.1364898 (5779)
5824:	learn: 0.0560173	test: 0.1364959	best: 0.1364898 (5779)
5825:	le

In [81]:
params = {'iterations': 6000,
          'learning_rate': 0.005,
          'depth': 4,
          'l2_leaf_reg': 1,
          'eval_metric':'RMSE',
          'early_stopping_rounds': 200,
          'verbose': 200,
          'random_seed': 42}
         
cat_f = CatBoostRegressor(**params)
cat_model_f = cat_f.fit(X_train,y_train,
                     eval_set = (X_val,y_val),
                     plot=True,
                     verbose = False)

catf_pred = cat_model_f.predict(X_val)
catf_score = rmse(y_val, catf_pred)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [82]:
catf_score

0.1085100462156005

In [83]:
test_pred = cat_f.predict(test)
submission = pd.DataFrame(test_id, columns = ['Id'])
test_pred = np.expm1(test_pred)
submission['SalePrice'] = test_pred 
submission.head()

,Id,SalePrice
0,1461,126984.852453
1,1462,158260.365041
2,1463,182676.276110
3,1464,190776.061675
4,1465,182630.989226


In [84]:
submission.to_csv("result.csv", index = False)